#### IMDb(Internet Movie Database)を使った感想のPositive,Negative分類

##### 使用するアーキテクチャ
- RNN
- LSTM

In [147]:
import random
import numpy as np
import string
import re
from collections import Counter
from typing import List
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import torchtext
from torchtext import data
from torchtext import datasets
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer
from sklearn.metrics import f1_score
from tqdm import tqdm

seed = 1234
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [148]:
class Config():
    batch_size=128
    n_epoch=20
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    emb_dim = 100
    hid_dim = 50

    checkpoint_path='model/IMDB.pt'
config=Config()

print(config.device)

cuda:0


In [149]:
def torch_log(x):
    return torch.log(torch.clamp(x,min=1e-10))

In [150]:
train_iter = datasets.IMDB(split='train')

train_iter, valid_iter = train_iter.random_split(
    weights={"train": 0.8, "valid": 0.2},
    seed=seed,
    total_length=len(list(train_iter)),
)

In [151]:
#tokenizerでインスタンス化
tokenizer = get_tokenizer("basic_english")

#counterでインスタンス化
counter = Counter()

#train_iterのlineをtokenizerを使ってcounterに加える
for label, line in train_iter:
    counter.update(tokenizer(line))

#vocabメソッドでvocabularyをインスタンス化
vocabulary = vocab(
    counter,
    min_freq=25,
    specials=('<unk>', '<PAD>', '<BOS>', '<EOS>')
)
# <unk>をデフォルトに設定することにより，min_freq回以上出てこない単語は<unk>になる
vocabulary.set_default_index(vocabulary['<unk>'])

word_num = len(vocabulary)

print(f"単語種数: {word_num}")
print(*vocabulary.get_itos()[:100], sep=', ')

単語種数: 9937
<unk>, <PAD>, <BOS>, <EOS>, i, am, curious, yellow, is, a, and, pretentious, steaming, pile, ., it, doesn, ', t, matter, what, one, s, political, views, are, because, this, film, can, hardly, be, taken, seriously, on, any, level, as, for, the, claim, that, frontal, male, nudity, an, automatic, ,, isn, true, ve, seen, films, with, granted, they, only, offer, some, fleeting, but, where, ?, nowhere, don, exist, same, goes, those, crappy, cable, shows, swinging, in, not, sight, indie, movies, like, brown, bunny, which, we, re, treated, to, site, of, vincent, johnson, trace, pink, visible, chloe, before, crying, (, or, ), matters


In [152]:
def text_transform(_text,max_length=256):
    text=[vocabulary[token] for token in tokenizer(_text)][:max_length-2]
    text=[vocabulary['<BOS>']]+text+[vocabulary['<EOS>']]

    return text,len(text)

def collate_batch(batch):
    label_list,text_list,len_seq_list=[],[],[]

    for _label,_text in batch:
    # torchtext==0.15.1からはnegativeは1，positiveは2なので，-1して{0, 1}にする
        label_list.append(_label-1)

        processed_test,len_seq=text_transform(_text)
        text_list.append(torch.tensor(processed_test))
        len_seq_list.append(len_seq)

    return torch.tensor(label_list),pad_sequence(text_list, padding_value=1).T, torch.tensor(len_seq_list)



In [153]:
train_dataloader=DataLoader(
    list(train_iter),
    batch_size=config.batch_size,
    shuffle=True,
    collate_fn=collate_batch
)

valid_dataloader=DataLoader(
    list(valid_iter),
    batch_size=config.batch_size,
    shuffle=False,
    collate_fn=collate_batch
)

#### LSTMとRNNで同じ訓練ループなので関数化する

In [154]:
class EarlyStopper:
    def __init__(self, verbose=True, path=config.checkpoint_path, patience=1):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.__early_stop = False
        self.val_f1_score = -np.Inf
        self.path = path
        
        
    @property
    def early_stop(self):
        return self.__early_stop

    def update(self, val_f1_score, model):
        if self.best_score is None:
            self.best_score = val_f1_score
            self.save_checkpoint(model, val_f1_score)
        elif val_f1_score < self.best_score:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.__early_stop = True
        else:
            self.best_score = val_f1_score
            self.save_checkpoint(model, val_f1_score)
            self.counter = 0
    
    def save_checkpoint(self, model, val_f1_score):
        if self.verbose:
            print(f'Validation f1score increased ({self.val_f1_score:.6f} --> {val_f1_score:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_f1_score = val_f1_score
        
    def load_checkpoint(self, model):
        if self.verbose:
            print(f'Loading model from last checkpoint with validation f1score {self.val_f1_score:.6f}')
        model.load_state_dict(torch.load(self.path))
        return model

early_stopping=EarlyStopper(patience=7,verbose=True)

In [155]:
from collections import OrderedDict
def train(net,optimizer,n_epoch=config.n_epoch):
    for epoch in range(n_epoch):
        losses_train=[]
        losses_valid=[]

        net.train()
        n_train=0

        with tqdm(train_dataloader) as pbar_epoch:
            for label,line,len_seq in pbar_epoch:
                net.zero_grad()

                t=label.to(config.device)
                x=line.to(config.device)
                len_seq.to(config.device)

                h=net(x,torch.max(len_seq),len_seq)
                y=torch.sigmoid(h).squeeze()#テンソル配列からサイズが１の次元を消去
                loss=-torch.mean(t*torch_log(y)+(1-t)*torch_log(1-y))

                loss.backward()  # 誤差の逆伝播

                optimizer.step()  # パラメータの更新

                losses_train.append(loss.tolist())

                n_train += t.size()[0]

                pbar_epoch.set_postfix(OrderedDict(train_loss=np.mean(losses_train)))
            # Valid
            t_valid = []
            y_pred = []
            net.eval()
        with tqdm(valid_dataloader) as pbar_epoch:
            for label, line, len_seq in pbar_epoch:

                t = label.to(config.device) # テンソルをGPUに移動
                x = line.to(config.device)
                len_seq.to(config.device)

                h = net(x, torch.max(len_seq), len_seq)
                y = torch.sigmoid(h).squeeze()

                loss = -torch.mean(t*torch_log(y) + (1 - t)*torch_log(1 - y))

                pred = y.round().squeeze()  # 0.5以上の値を持つ要素を正ラベルと予測する

                t_valid.extend(t.tolist())
                y_pred.extend(pred.tolist())

                losses_valid.append(loss.tolist())
                val_f1_score=f1_score(t_valid, y_pred, average='macro')

            early_stopping.update(val_f1_score,model=net)

            print('EPOCH: {}, Train Loss: {:.3f}, Valid Loss: {:.3f}, Validation F1: {:.3f}'.format(
                epoch+1,
                np.mean(losses_train),
                np.mean(losses_valid),
                f1_score(t_valid, y_pred, average='macro')
            ))
            pbar_epoch.set_postfix(OrderedDict(valid_loss=np.mean(losses_valid),f1_score=f1_score(t_valid, y_pred, average='macro')))
            if early_stopping.early_stop:
                print('Early Stopping!')
                break

    net=early_stopping.load_checkpoint(net)

#### Embedding層の実装
- 単語を離散的なIDから連続的なベクトルに変換する
- 埋め込み行列も学習に使われるパラメータで単語間の類似度を示す
- 埋め込み行列はランダムに初期化する

以下の仮想的な状況を考える<br>
辞書: {'猫': 0, 'は': 1, '魚': 2, 'が': 3, '好き': 4, 'です': 5}

V (埋め込み行列):<br>
0: [0.1, 0.3, 0.2]  # 「猫」の埋め込みベクトル<br>
1: [0.2, 0.1, 0.3]  # 「は」の埋め込みベクトル<br>
2: [0.3, 0.2, 0.1]  # 「魚」の埋め込みベクトル<br>
3: [0.1, 0.2, 0.3]  # 「が」の埋め込みベクトル<br>
4: [0.2, 0.3, 0.1]  # 「好き」の埋め込みベクトル<br>
5: [0.3, 0.1, 0.2]  # 「です」の埋め込みベクトル<br>

これに対して以下の文があったとすると<br>
x (単語ID):<br>
文1: [0, 1, 2, 3, 4, 5]  # 「猫は魚が好きです」<br>
文2: [2, 1, 0, 3, 4, 5]  # 「魚は猫が好きです」<br>

出力のテンソルは以下のようになる<br>

出力テンソル:<br>
文1: [[0.1, 0.3, 0.2], [0.2, 0.1, 0.3], [0.3, 0.2, 0.1], [0.1, 0.2, 0.3], [0.2, 0.3, 0.1], [0.3, 0.1, 0.2]]  # 「猫は魚が好きです」<br>
文2: [[0.3, 0.2, 0.1], [0.2, 0.1, 0.3], [0.1, 0.3, 0.2], [0.1, 0.2, 0.3], [0.2, 0.3, 0.1], [0.3, 0.1, 0.2]]  # 「魚は猫が好きです」

In [156]:
class Embedding(nn.Module):
    def __init__(self,emb_dim,vocab_size):
        super().__init__()
        self.embedding_matrix=nn.Parameter(torch.rand((vocab_size,emb_dim),
                                                    dtype=torch.float))
#Embeddingの実行
    def forward(self,x):
        return F.embedding(x,self.embedding_matrix)

In [157]:
class RNN(nn.Module):
    def __init__(self, in_dim, hid_dim):
        super().__init__()
        self.hid_dim = hid_dim
        glorot = 6 / (in_dim + hid_dim*2)
        self.W = nn.Parameter(torch.tensor(np.random.uniform(
                        low=-np.sqrt(glorot),
                        high=np.sqrt(glorot),
                        size=(in_dim + hid_dim, hid_dim)
                    ).astype('float32')))
        self.b = nn.Parameter(torch.tensor(np.zeros([hid_dim]).astype('float32')))

    def function(self, h, x):
        return torch.tanh(torch.matmul(torch.cat([h, x], dim=1), self.W) + self.b)

    def forward(self, x, len_seq_max=0, init_state=None):
        x = x.transpose(0, 1)  # 系列のバッチ処理のため、次元の順番を「系列、バッチ」の順に入れ替える
        state = init_state

        if init_state is None:  # 初期値を設定しない場合は0で初期化する
            state = torch.zeros((x[0].size()[0], self.hid_dim)).to(x.device)

        size = list(state.unsqueeze(0).size())
        size[0] = 0
        output = torch.empty(size, dtype=torch.float).to(x.device)  # 一旦空テンソルを定義して順次出力を追加する

        if len_seq_max == 0:
            len_seq_max = x.size(0)
        for i in range(len_seq_max):
            state = self.function(state, x[i])
            output = torch.cat([output, state.unsqueeze(0)])  # 出力系列の追加
        return output

In [158]:
class SequenceTaggingNet(nn.Module):
    def __init__(self, word_num, emb_dim, hid_dim):
        super().__init__()
        self.emb = Embedding(emb_dim, word_num)
        self.rnn = RNN(emb_dim, hid_dim)
        self.linear = nn.Linear(hid_dim, 1)

    def forward(self, x, len_seq_max=0, len_seq=None, init_state=None):
        h = self.emb(x)
        h = self.rnn(h, len_seq_max, init_state)
        if len_seq is not None:
            # 系列が終わった時点での出力を取る必要があるので len_seq を元に集約する
            h = h[len_seq - 1, list(range(len(x))), :]
        else:
            h = h[-1]
        y = self.linear(h)
        return y
    
# RNNのモジュールを使った場合のモデルを構築する
    
class SeqenceTaggingNet2(nn.Module):
    def __init__(self,word_num,emb_dim,hid_dim):
        self.emb=Embedding(emb_dim,word_num)
        self.rnn=nn.RNN(emb_dim,hid_dim,1,batch_first=True)
        self.linear=nn.Linear(hid_dim,1)

    def forward(self, x, len_seq_max=0, len_seq=None, init_state=None):
        h = self.emb(x)
        if len_seq_max > 0:
            h, _ = self.rnn(h[:, 0:len_seq_max, :], init_state)
        else:
            h, _ = self.rnn(h, init_state)
        h = h.transpose(0, 1)
        if len_seq is not None:
            # 系列が終わった時点での出力を取る必要があるので len_seq を元に集約する
            h = h[len_seq - 1, list(range(len(x))), :]
        else:
            h = h[-1]
        y = self.linear(h)

        return y

In [159]:
net = SequenceTaggingNet(word_num, config.emb_dim, config.hid_dim)
net.to(config.device)

optimizer = optim.Adam(net.parameters())

train(net, optimizer)

  1%|▏         | 2/157 [00:00<00:17,  8.71it/s, train_loss=0.752]

100%|██████████| 40/40 [00:02<00:00, 18.16it/s]


Validation f1score increased (-inf --> 0.462919).  Saving model ...
EPOCH: 1, Train Loss: 0.692, Valid Loss: 0.687, Validation F1: 0.463


100%|██████████| 40/40 [00:02<00:00, 18.51it/s]


Validation f1score increased (0.462919 --> 0.596929).  Saving model ...
EPOCH: 2, Train Loss: 0.656, Valid Loss: 0.638, Validation F1: 0.597


...

EarlyStopping counter: 5 out of 7
EPOCH: 15, Train Loss: 0.146, Valid Loss: 1.111, Validation F1: 0.640


100%|██████████| 40/40 [00:02<00:00, 18.63it/s]


EarlyStopping counter: 6 out of 7
EPOCH: 16, Train Loss: 0.122, Valid Loss: 1.187, Validation F1: 0.642


100%|██████████| 40/40 [00:02<00:00, 18.09it/s]

EarlyStopping counter: 7 out of 7
EPOCH: 17, Train Loss: 0.101, Valid Loss: 1.320, Validation F1: 0.631
Early Stopping!
Loading model from last checkpoint with validation f1score 0.726960


### LSTMを用いた実装

- 入力ゲート: $\hspace{20mm}\boldsymbol{i}_t = \mathrm{\sigma} \left(\boldsymbol{W}_i \left[\begin{array}{c} \boldsymbol{x}_t \\ \boldsymbol{h}_{t-1} \end{array}\right] + \boldsymbol{b}_i\right)$
- 忘却ゲート: $\hspace{20mm}\boldsymbol{f}_t = \mathrm{\sigma} \left(\boldsymbol{W}_f \left[\begin{array}{c} \boldsymbol{x}_t \\ \boldsymbol{h}_{t-1} \end{array}\right] + \boldsymbol{b}_f\right)$  
- 出力ゲート: $\hspace{20mm}\boldsymbol{o}_t = \mathrm{\sigma} \left(\boldsymbol{W}_o \left[\begin{array}{c} \boldsymbol{x}_t \\ \boldsymbol{h}_{t-1} \end{array}\right] + \boldsymbol{b}_o\right)$  
- セル:　　　 $\hspace{20mm}\boldsymbol{c}_t = \boldsymbol{f}_t \odot \boldsymbol{c}_{t-1} + \boldsymbol{i}_t \odot \tanh \left(\boldsymbol{W}_c \left[\begin{array}{c} \boldsymbol{x}_t \\ \boldsymbol{h}_{t-1} \end{array}\right] + \boldsymbol{b}_c\right)$
- 隠れ状態: 　$\hspace{20mm}\boldsymbol{h}_t = \boldsymbol{o}_t \odot \tanh \left(\boldsymbol{c}_t \right)$

In [160]:
class SequenceTaggingNet4(nn.Module):
    def __init__(self, word_num, emb_dim, hid_dim):
        super().__init__()
        self.emb = nn.Embedding(word_num, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim, 1, batch_first=True)  # nn.LSTMの使用
        self.linear = nn.Linear(hid_dim, 1)
    
    def forward(self, x, len_seq_max=0, len_seq=None, init_state=None):
        h = self.emb(x)
        if len_seq_max > 0:
            h, _ = self.lstm(h[:, 0:len_seq_max, :], init_state)
        else:
            h, _ = self.lstm(h, init_state)
        h = h.transpose(0, 1)
        if len_seq is not None:
            h = h[len_seq - 1, list(range(len(x))), :]
        else:
            h = h[-1]
        y = self.linear(h)
        
        return y

In [161]:

early_stopping=EarlyStopper(patience=7,verbose=True,path='./model/IMBD_LSTM.pt')
net = SequenceTaggingNet4(word_num, config.emb_dim, config.hid_dim)
net.to(config.device)
optimizer = optim.Adam(net.parameters())

train(net, optimizer)

100%|██████████| 40/40 [00:01<00:00, 31.04it/s]


Validation f1score increased (-inf --> 0.618530).  Saving model ...
EPOCH: 1, Train Loss: 0.674, Valid Loss: 0.635, Validation F1: 0.619


100%|██████████| 40/40 [00:01<00:00, 30.92it/s]


Validation f1score increased (0.618530 --> 0.710577).  Saving model ...
EPOCH: 2, Train Loss: 0.599, Valid Loss: 0.585, Validation F1: 0.711


100%|██████████| 40/40 [00:01<00:00, 30.54it/s]


...

EarlyStopping counter: 5 out of 7
EPOCH: 17, Train Loss: 0.404, Valid Loss: 0.432, Validation F1: 0.808


100%|██████████| 40/40 [00:01<00:00, 31.39it/s]


EarlyStopping counter: 6 out of 7
EPOCH: 18, Train Loss: 0.301, Valid Loss: 0.402, Validation F1: 0.839


100%|██████████| 40/40 [00:01<00:00, 30.77it/s]

EarlyStopping counter: 7 out of 7
EPOCH: 19, Train Loss: 0.249, Valid Loss: 0.430, Validation F1: 0.834
Early Stopping!
Loading model from last checkpoint with validation f1score 0.851991
